In [1]:
import numpy as np 
import pandas as pd 

## Loading Data

In [3]:
from datasets import load_dataset
dataset = pd.read_csv("doc_query_pairs.train.tsv", sep='\t')


In [4]:
dataset.head()

,The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.,)what was the immediate impact of the success of the manhattan project?
0,The approach is based on a theory of justice t...,_________ justice is designed to repair the ha...
1,"Colorâurine can be a variety of colors, most...",what color is amber urine
2,Inborn errors of bile acid synthesis can produ...,is autoimmune hepatitis a bile acid synthesis ...
3,The word convict here (elegcw /elegxo) means t...,elegxo meaning
4,In-home tutors can earn anywhere from $10 to $...,how much does an average person make for tutoring


In [5]:
dataset.columns = ['Text', 'Query']


In [6]:
dataset.head()

,Text,Query
0,The approach is based on a theory of justice t...,_________ justice is designed to repair the ha...
1,"Colorâurine can be a variety of colors, most...",what color is amber urine
2,Inborn errors of bile acid synthesis can produ...,is autoimmune hepatitis a bile acid synthesis ...
3,The word convict here (elegcw /elegxo) means t...,elegxo meaning
4,In-home tutors can earn anywhere from $10 to $...,how much does an average person make for tutoring


In [7]:
from datasets import Dataset

# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(dataset)

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import BartTokenizer

# Load tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

# Load and prepare the DataFrame
df = pd.read_csv("doc_query_pairs.train.tsv", sep='\t')
df.columns = ['Text', 'Query']  # Adjust this if your file has different column headers

# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Function to preprocess the data

def preprocess_function(examples):
# Tokenize both the input texts and the queries in one call
    model_inputs = tokenizer(
        examples["Text"],
        text_target=examples["Query"],
        max_length=1024,
        truncation=True,
        padding="max_length",
        return_tensors="np"  # Return as NumPy arrays if you're not immediately loading to a model, else use "pt" for PyTorch
    )
    
    # Correct extraction of labels
    # The tokenizer outputs are returned in a dictionary with input_ids, attention_mask, and labels (since text_target is used)
    # The labels here are the input_ids for the target texts
    model_inputs["labels"] = model_inputs.pop('labels')
    return model_inputs

# Apply preprocessing
dataset = dataset.map(preprocess_function, batched=True)

# Split the dataset into training and test sets
train_test_split = dataset.train_test_split(test_size=0.1) 
dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']  
})


In [9]:
dataset.save_to_disk("dataset/")

Saving the dataset (0/14 shards):   0%|          | 0/479475 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/53275 [00:00<?, ? examples/s]

## Model Training

In [2]:
from datasets import load_from_disk

# Load the dataset from disk
dataset = load_from_disk("dataset/")

In [3]:
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments, BartTokenizer
import torch

# Specify a new cache directory with sufficient space
saved_model_path = "/data/bhaskarc/nirmal/IR/TinyBert/bart_base2/model/"
saved_tokenizer_path = "/data/bhaskarc/nirmal/IR/TinyBert/bart_base2/tokenizer/"

# Load the model with a specified cache directory
model = BartForConditionalGeneration.from_pretrained(saved_model_path)

tokenizer = BartTokenizer.from_pretrained(saved_tokenizer_path)

# Check if a GPU is available and set the device accordingly
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)  # Ensure the model is on the correct device
print(f"Model is using device: {device}")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=18000,  # Evaluation and saving happens every 500 steps
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
    save_total_limit=3,  # Only last 3 models are saved
    num_train_epochs=2,
    report_to="none",  # Or 'wandb' for Weights & Biases 
    fp16=True
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
)

2024-11-25 15:38:19.840493: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-25 15:38:19.854646: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732529299.872152 3308903 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732529299.877553 3308903 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-25 15:38:19.895528: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Model is using device: cuda


/home/bhaskarc/miniconda3/envs/kalp_gpu/lib/python3.9/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [4]:
# Start training
trainer.train()

Step,Training Loss,Validation Loss
18000,0.012000,0.014976
36000,0.012700,0.014512
54000,0.014100,0.013844
72000,0.012200,0.013619
90000,0.011600,0.013351
108000,0.011600,0.013087


TrainOutput(global_step=119868, training_loss=0.012282160976394764, metrics={'train_runtime': 61601.9595, 'train_samples_per_second': 15.567, 'train_steps_per_second': 1.946, 'total_flos': 5.847025084740403e+17, 'train_loss': 0.012282160976394764, 'epoch': 1.999983315119005})

In [5]:
# Saving the model manually
model.save_pretrained('bart_base3/model')
tokenizer.save_pretrained('bart_base3/tokenizer')

('bart_base3/tokenizer/tokenizer_config.json',
 'bart_base3/tokenizer/special_tokens_map.json',
 'bart_base3/tokenizer/vocab.json',
 'bart_base3/tokenizer/merges.txt',
 'bart_base3/tokenizer/added_tokens.json')

In [25]:
# Print some sample labels and inputs to check alignment
for i in range(5):
    print("Input IDs:", encoded_dataset['train'][i]['input_ids'])
    print("Labels:", encoded_dataset['train'][i]['labels'])


Input IDs: [0, 574, 29876, 8488, 7810, 4759, 22482, 4, 7810, 4759, 22482, 16, 10, 3286, 11, 10347, 1073, 4317, 18, 47898, 11170, 11, 20, 23026, 44936, 5855, 6, 8, 5, 317, 147, 37, 16, 6475, 7, 8386, 7736, 21194, 88, 1241, 10524, 29860, 994, 4, 404, 472, 386, 49, 3251, 259, 6, 19, 5, 9794, 16455, 840, 1069, 2987, 11, 7810, 4759, 22482, 4, 1437, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Labels: [0, 12196, 232, 16, 5, 7047, 39573, 2987, 11, 2569, 4759, 22482, 2034,